In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [12]:
years = [
    "january-2012", "february-2012", "march-2012", "april-2012",
    "may-2012", "june-2012", "july-2012", "august-2012",
    "september-2012", "october-2012", "november-2012", "december-2012"
]


base_url = "https://weatherandclimate.com"

pages = []
next_years = []

# Generate URLs for each month in the current year
for year in years:
    url = f"{base_url}/{year}"
    pages.append(url)

    # Extract the year from the month string
    current_year = int(year.split('-')[1])
    next_year = current_year + 1

    # Generate URLs for the same months in the next year
    next_year_url = f"{base_url}/{year.split('-')[0]}-{next_year}"
    next_years.append(next_year_url)




In [43]:
all_data=pd.DataFrame()
all_headers=[]

counties = ['kisii','kisumu','kitui','kwale','laikipia', 'lamu','machakos',
            'makueni','mandera','marsabit', 'meru','migori','mombasa','murang-a','nairobi-city','nakuru'] 
all_pages=[]

for county in counties:
    pages = [f"{base_url}/{county}/{year}" for year in years]
    all_pages.append(pages)

for page in all_pages:
    for j in page:
        response=requests.get(j)
        data=response.text
        soup = BeautifulSoup(data, "html.parser")

        table=soup.find('table',class_='tb8')

    #extract headers
        headers=table.find('tr')
        header=[j.get_text(strip=True) for j in headers.find_all('th')]
        all_headers.append(header)

        #extract rows
        for row in table.find_all('tr')[1:]:
            rows=[k.get_text(strip=True) for k in row.find_all('td')]
            temperature_data = {
                'Max Temperature': None,
                'Avg Temperature': None,
                'Min Temperature': None,
                'Dew Point': None,
                'Precipitation': None,
                'Wind': None,
                'Sea Level Pressure': None
            }
            if len(rows) > 1:  # Ensure there are enough cells in the row
                    label = rows[0]
                    value = rows[1]
                    if 'Max Temperature' in label:
                        temperature_data['Max Temperature'] = value
                    elif 'Avg Temperature' in label:
                        temperature_data['Avg Temperature'] = value
                    elif 'Min Temperature' in label:
                        temperature_data['Min Temperature'] = value
                    elif 'Dew Point' in label:
                        temperature_data['Dew Point'] = value
                    elif 'Precipitation' in label:
                        temperature_data['Precipitation'] = value
                    elif 'Wind' in label:
                        temperature_data['Wind'] = value
                    elif 'Sea Level Pressure' in label:
                        temperature_data['Sea Level Pressure'] = value

            # Append the extracted data to the DataFrame
            df = pd.DataFrame({
                'County': [county],
                'Month': [year.strip('-')[0:5]],  # Assuming 'year' is equivalent to 'Month'
                'Year': [year],
                'Max Temperature': [temperature_data.get('Max Temperature')],
                'Avg Temperature': [temperature_data.get('Avg Temperature')],
                'Min Temperature': [temperature_data.get('Min Temperature')],
                'Dew Point': [temperature_data.get('Dew Point')],
                'Precipitation': [temperature_data.get('Precipitation')],
                'Wind': [temperature_data.get('Wind')],
                'Sea Level Pressure': [temperature_data.get('Sea Level Pressure')]
            })
                
            all_data = pd.concat([all_data, df], ignore_index=True)




In [55]:
all_data.sample(5)

,County,Month,Year,Max Temperature,Avg Temperature,Min Temperature,Dew Point,Precipitation,Wind,Sea Level Pressure
2185,nakuru,2012,2012,None,23.0°C (73.4°F),None,None,None,None,None
557,nakuru,2012,2012,None,None,None,None,None,None,None
2445,nakuru,2012,2012,None,21.0°C (69.8°F),None,None,None,None,None
2424,nakuru,2012,2012,None,None,None,None,14.6mm | 0.57in,None,None
1935,nakuru,2012,2012,None,None,None,None,None,None,None


In [57]:
all_data.to_csv("climate_data")